Veri temizleme, veri biliminin önemli bir parçasıdır. 

Bu derste, verilerinizi daha hızlı analiz edebilmek için en yaygın veri temizleme sorunlarından bazılarını nasıl çözeceğiniz ile ilgili alıştırmalar mevcut.

Bu notebookta, eksik değerlerle nasıl başa çıkılacağına bakacağız.

# Öncelikle Verilere göz atalım

Yapmamız gereken ilk şey, kullanacağımız kütüphaneleri ve veri setini yüklemek.

Amerikan Futbolu oyunlarında meydana gelen olayların bir veri setini kullanacağız.


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day3-FeatureSelection/notebooks"
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART1/Day3-FeatureSelection/notebooks"
DATASET_PATH = ROOT_DIR + "/datasets/"

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
nfl_data = pd.read_csv(DATASET_PATH + "/NFL_Play_by_Play_2009-2017_v4.csv")

# set seed for reproducibility
np.random.seed(0) 

Yeni bir veri seti aldığınızda yapılacak ilk şey, verilere göz atmaktır. Bu, her şeyin doğru bir şekilde okunduğunu görmenizi sağlar ve verilerde neler olduğu hakkında bir fikir verir. Bu durumda, `NaN` veya `None` ile temsil edilecek herhangi bir eksik değer olup olmadığını görelim.


In [ ]:
# look at the first five rows of the nfl_data file. 
# I can see a handful of missing data already!
nfl_data.head()

Evet, bazı eksik değerler var gibi görünüyor.

# Kaç tane eksik veri noktamız var?

Tamam, artık bazı eksik değerlerimiz olduğunu biliyoruz. Bakalım her sütunda kaç tane var.


In [ ]:
# get the number of missing data points per column
missing_values_count = nfl_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Bu çok gibi görünüyor! Bu sorunun ölçeğini daha iyi anlamamız için veri kümemizdeki değerlerin yüzde kaçının eksik olduğunu görmek yardımcı olabilir:


In [ ]:
# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)

total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

Bu veri kümesindeki hücrelerin neredeyse dörtte biri boş! Bir sonraki adımda, değerleri eksik olan bazı sütunlara daha yakından bakacağız ve onlarla neler olabileceğini anlamaya çalışacağız.

# Verilerin neden eksik olduğunu anlayın
 
Bu, veri biliminde "veri sezgisi" olarak adlandırılan, yani "verilerinize gerçekten bakmak ve bunun neden böyle olduğunu ve bunun nasıl olacağını anlamaya çalışmak" anlamına gelen kısmına girdiğimiz noktadır. Eksik değerlerle başa çıkmak için, değerin neden eksik olduğunu anlamak için sezginizi kullanmanız gerekir. Bunu çözmenize yardımcı olması için kendinize sorabileceğiniz en önemli sorulardan biri şudur:

> **Bu değer kaydedilmediği için mi yoksa var olmadığı için mi eksik?**

Bir değer olmadığı için eksikse (çocuğu olmayan birinin en büyük çocuğunun boyu gibi), o zaman ne olabileceğini tahmin etmeye çalışmak mantıklı değildir. Bu değerleri muhtemelen `NaN` olarak tutmak istersiniz. Öte yandan, bir değer kaydedilmediği için eksikse, o sütun ve satırdaki diğer değerlere dayanarak ne olabileceğini tahmin etmeye çalışabilirsiniz. Buna **imputation** denir ve bundan sonra nasıl yapılacağını öğreneceğiz! :)

Bir örnek üzerinden çalışalım. `nfl_data` dataframe'indeki eksik değerlerin sayısına baktığımızda, "TimesSec" sütununun içinde birçok eksik değer olduğunu fark ediyoruz:


In [ ]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

[Belgelere](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016) bakarak, bu sütunda oyun yapıldığında oyunda kalan saniye sayısı hakkında bilgi olduğunu görebiliyorum. Bu, bu değerlerin, var olmadıkları için değil, kaydedilmedikleri için muhtemelen eksik olduğu anlamına gelir. Bu nedenle, onları NA'lar olarak bırakmak yerine, ne olmaları gerektiğini tahmin etmeye çalışmak bizim için mantıklı olacaktır.

Öte yandan, "PenalizedTeam" gibi birçok eksik alanı olan başka alanlar da var. Ancak bu durumda oyunda penaltı olmaması durumunda, *hangi takımın ceza aldığını söylemek anlamsızdır. Bu sütun için, onu boş bırakmak veya "neither" gibi üçüncü bir değer eklemek ve bunu NA'ları değiştirmek için kullanmak daha mantıklı olacaktır.

Çok dikkatli veri analizi yapıyorsanız, bu eksik değerleri doldurmak için en iyi stratejiyi bulmak için her sütuna ayrı ayrı bakmanız gerekir. 

# Eksik değerleri kaldırma

Aceleniz varsa veya değerlerinizin neden eksik olduğunu anlamak için bir nedeniniz yoksa, sahip olduğunuz seçeneklerden biri eksik değerler içeren satırları veya sütunları kaldırmaktır. (Not: Bu yaklaşımı önemli projeler için önerilmez! Veri kümenizi gerçekten tanımak için genellikle verilerinizi gözden geçirmek ve eksik değerleri olan tüm sütunlara tek tek bakmak için zaman ayırmaya değer.)

Eksik değerlere sahip satırları bırakmak istediğinizden eminseniz, pandas'ın bunu yapmanıza yardımcı olacak kullanışlı bir işlevi `dropna()` vardır. NFL veri setimizde deneyelim!

In [ ]:
### !!!!!!!!!
# remove all the rows that contain a missing value
nfl_data.dropna()

Tüm verilerimizi kaldırmış gibi görünüyor! 😱 Bunun nedeni, veri kümemizdeki her satırın en az bir eksik değere sahip olmasıdır. Bunun yerine en az bir eksik değeri olan tüm *sütunları* kaldırırsak daha iyi olabilir.


In [ ]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

In [ ]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Oldukça fazla veri kaybettik, ancak bu noktada tüm `NaN`leri verilerimizden başarıyla kaldırdık.

# Eksik değerleri otomatik olarak doldurma

Başka bir seçenek de eksik değerleri denemek ve doldurmaktır. Sonraki bit için, iyi yazdırılabilmesi için NFL verilerinin küçük bir alt bölümünü alalım.


In [ ]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

Bir veri çerçevesindeki eksik değerleri bizim için doldurmak için Pandas'ın `fillna()` fonksiyonunu kullanabiliriz. Elimizdeki seçeneklerden biri, `NaN` değerlerinin neyle değiştirilmesini istediğimizi belirtmektir. Burada tüm `NaN` değerlerini 0 ile değiştirmek istediğimi söylüyorum.


In [ ]:
# replace all NA's with 0
subset_nfl_data.fillna(0)

Ayrıca biraz daha bilgili olabilir ve eksik değerleri, aynı sütunda hemen ardından gelen değerle değiştirebilirim. (Bu, gözlemlerin bir tür mantıksal sıraya sahip olduğu veri kümeleri için çok anlamlıdır.)


In [ ]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the remaining na's with 0
subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)